In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
import matplotlib.pyplot as plt

## Common Data Cleaning Tasks

* Handling missing data
* Handling duplicates
* Detecting and filtering outliers
* Discretize and bin
* Creating dummy variables, combine categories, other encodings
* Changing data types
* Unpacking (extract info from a column)
* Tidying data

*Note:*  Regular expressions and string manipulation methods are often useful in data cleaning, particularly when there is some form of text data.

In [ ]:
df = pd.DataFrame([[1., 6.5, 3.0, 0.5], [1., np.nan, 3, 1.5],
                     [np.nan, np.nan, np.nan, np.nan], [np.nan, 6.5, np.nan, 3.5],
                     [np.nan, 6.5, 3.0, 3.5]],
                 columns=['A','B','C','D'])

In [ ]:
df

## Missing data

Data that is to be used in a machine or statistical learning model needs to be absent of missing values.  Below are some ways to handle missing values in pandas.  The "correct" way to treat missing values will be context specific.  Analyses might compare results from how missing values were handled differently.  Often, missing values are filled using `Scikit-Learn` so the process can be built into a pipeline. 

1. **Detecting Missing Values**:
    - `isna()` or `isnull()`: Returns a DataFrame or Series indicating which values are missing.
    - `notna()` or `notnull()`: Opposite of `isna()`, indicates which values are not missing.
  
2. **Removing Missing Values**:
    - `dropna()`: 
        - Remove missing values.
        - Can specify `axis` (rows or columns).
        - Can use `how` argument to specify 'any' (default) or 'all' to determine when rows/columns are dropped.
        - Can use `subset` argument to specify to only consider certain  columns or rows

3. **Filling Missing Values**:
    - `fillna()`:
        - Fill missing values with a specific value.
        - Use `method` argument to specify method ('ffill' for forward fill, 'bfill' for backward fill).
        - Can fill using the mean, median, or mode.
        - Can also use interpolation methods.

4. **Replace Missing Values**:
    - `replace()`: 
        - Replace specified values with another value (can be used for values other than NaN too).

5. **Using an Indicator**:
    - Add a new column to indicate where values are missing: 
        ```python
        df['missing_col_indicator'] = df['col'].isnull()
        ```

6. **Group-wise Handling**:
    - Use `groupby` along with `fillna()` to fill missing values based on some group properties:
        ```python
        df['col'] = df.groupby('group_col')['col'].transform(lambda x: x.fillna(x.mean()))
        ```

7. **Forward and Backward Filling**:
    - `ffill()` or `pad()`: Fill values forward.
    - `bfill()` or `backfill()`: Fill values backward.

8. **Limiting Fill or Drop Actions**:
    - Using the `limit` parameter in methods like `dropna()` and `fillna()` to restrict the **number** of rows/columns affected.
    - Using the `subset` argument in methods like `dropna()` and `fillna()` to specify the actual row/columns affected.



In [ ]:
df

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(subset=['D', 'C'], how='all')

In [ ]:
df.fillna(value=df.mean())

In [ ]:
df

---

## Duplicates

Handling duplicate data is important, especially in data preprocessing, to ensure the accuracy of subsequent analyses. Here's a bullet list of possible ways to check for and handle duplicate data in pandas:

1. **Checking for Duplicates**:
    - `duplicated()`: Returns a Boolean series that indicates whether each row is a duplicate or not.

        ```python
        df = pd.DataFrame({'A': [1, 2, 2, 3, 3], 'B': [1, 1, 2, 2, 3]})
        print(df.duplicated())
        ```

2. **Counting Duplicates**:
    - Chain the `sum()` function after `duplicated()` to get the total count of duplicate rows.

        ```python
        print(df.duplicated().sum())
        ```

3. **Viewing Duplicates**:
    - Use boolean indexing to filter and see the actual duplicate rows.

        ```python
        print(df[df.duplicated()])
        ```

4. **Removing Duplicates**:
    - `drop_duplicates()`: Removes the duplicate rows from the DataFrame.

        ```python
        df_no_duplicates = df.drop_duplicates()
        ```

5. **Considering Specific Columns**:
    - `drop_duplicates(subset=column_name)`: Check for duplicates based on specific columns.

        ```python
        df_no_dup_in_A = df.drop_duplicates(subset='A')
        ```

6. **Keeping Specific Duplicates**:
    - The `keep` parameter in `drop_duplicates()`:
        - `keep='first'` (default): Keeps the first occurrence of the duplicate.
        - `keep='last'`: Keeps the last occurrence of the duplicate.
        - `keep=False`: Removes all occurrences of the duplicates.

        ```python
        df_removed_all = df.drop_duplicates(keep=False)
        ```

7. **Checking Duplicates in Columns**:
    - Sometimes you might want to check duplicates within a column.

        ```python
        print(df['A'].duplicated().sum())
        ```

8. **Removing Duplicates in Columns**:
    - To drop duplicate values in a specific column, you can use `drop_duplicates` with the `subset` argument.

        ```python
        df_no_dup_column = df.drop_duplicates(subset='A', keep='first')
        ```

9. **Resetting Index**:
    - After dropping duplicates, you might have gaps in your index. You can reset the index using `reset_index(drop=True)`.

        ```python
        df_reset = df_no_duplicates.reset_index(drop=True)
        ```

10. **Considering Data Order**:
    - When handling duplicates, the order of data can matter. Ensure you've sorted the data (if necessary) before addressing duplicates.


In [ ]:
df

In [ ]:
df.drop_duplicates(subset=['B','D'])

---

## Outliers

Handling outliers is essential as they can distort results and lead to incorrect conclusions. However, it is important to remember that not all outliers are bad or erroneous. Like many data cleaning steps, the appropriate way to handle outliers can depend on the problem.

Here are a few possible ways to check for and handle outliers in pandas:



1. **Visualization**:
    - Using plots can help in visually identifying outliers.
    - `boxplot()`: A box plot can reveal outliers as points beyond the "whiskers" of the box plot.

        ```python
        import matplotlib.pyplot as plt
        df = pd.DataFrame({'Values': [1, 2, 3, 50]})
        df.boxplot(column='Values')
        plt.show()
        ```

2. **Descriptive Statistics**:
    - `describe()`: Gives a summary of statistics, where the minimum and maximum values can provide insights about potential outliers.
    
        ```python
        print(df.describe())
        ```

3. **Z-Score**:
    - Measure the number of standard deviations a data point is from the mean.
    - Data points with a z-score greater than 3 or less than -3 are typically considered outliers.
        ```python
        from scipy.stats import zscore
        abs_z_scores = np.abs(zscore(df['Values']))
        outliers = df[abs_z_scores > 3]
        print(outliers)
        ```

4. **IQR (Interquartile Range)**:
    - IQR is the range between the first quartile (25th percentile) and the third quartile (75th percentile).
    - Data points outside 1.5 times the IQR below the first quartile or above the third quartile might be considered outliers.

        ```python
        Q1 = df['Values'].quantile(0.25)
        Q3 = df['Values'].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[((df['Values'] < (Q1 - 1.5 * IQR)) | (df['Values'] > (Q3 + 1.5 * IQR)))]
        print(outliers)
        ```

5. **Removing Outliers**:
    - After identifying outliers, you can drop them using boolean indexing.

        ```python
        filtered_df = df[(abs_z_scores < 3)]
        ```

6. **Capping Outliers**:
    - Instead of removing outliers, you can limit their value to a specific range (capping).

        ```python
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df['Values'] = np.where(df['Values'] > upper_bound, upper_bound, df['Values'])
        df['Values'] = np.where(df['Values'] < lower_bound, lower_bound, df['Values'])
        ```

7. **Transforming Data**:
    - Outliers can sometimes be managed by transforming data to fit into a more regular shape.
    - Common transformations include square root, log, and box-cox transformations.

        ```python
        df['Log_Values'] = np.log(df['Values'] + 1)
        ```


In [ ]:
np.random.seed(100)
data = pd.DataFrame(np.random.standard_normal((10, 4)))
data = data*3

In [ ]:
data

In [ ]:
data[data[0].abs() > 4]

In [ ]:
data[(data.abs() > 4).any(axis=1)]

In [ ]:
# replace outliers with .mask()

`.mask` is an application of the if-then idiom.  If "condition" is True, replace with "value".  There are also `inplace` and `axis` arguments.

`df.mask(condition, value)`

In [ ]:
data.quantile(.9)

In [ ]:
data_fixed = data.mask(data>4, data.quantile(.9), axis=1)

In [ ]:
data_fixed = data_fixed.mask(data< -4, data.quantile(.1), axis=1)

In [ ]:
data.mask(data.abs()>4, np.nan)

---

## Discretizing

Another way to handle outliers, is to discretize or bin a continuous/numeric variable.  Other reasons for binning include (not an exhaustive list):
- Data simplification
- Visualization
- Dealing with noise and variability
- Improving model performance
- Handling sparse data areas
- Ensuring privacy
- Etc. 

In [ ]:
s = data.iloc[:,0]

In [ ]:
s

In [ ]:
bins = [-10,-2,-1,1,2,10]

In [ ]:
pd.cut(s, bins)

In [ ]:
pd.cut(s,bins).value_counts()

In [ ]:
pd.cut(s,bins).value_counts()

In [ ]:
pd.cut(s,bins,labels=['very low','low','average','high','very high'])

---

## Dummy Variables

Creating dummy variables is also known as One-hot encoding.  It is a possible method for converting categorical variables into numbers that can be fed into a machine or statistical learning model. 


1. **Binary Categorical Data**:
    - For binary categorical data (e.g., "Yes" or "No"), you can represent it with one dummy variable (1 for "Yes" and 0 for "No").

2. **Multi-category Categorical Data**:
    - For a categorical variable with *n* distinct categories, you can represent it using *n-1* or *n* dummy variables, depending on the approach.
    - Why *n-1* or *n*? Using *n-1* dummy variables avoids the "dummy variable trap" or multicollinearity, where one variable can be predicted from the others. However, in some situations, like with certain machine learning algorithms or when clarity is needed, all *n* dummy variables might be used.

### Example:

Let's take an example where a feature "Color" in a dataset has three categories: "Red", "Green", and "Blue".

Using dummy variables, this can be represented as:

| Color | Red | Green | Blue |
|-------|-----|-------|------|
| Red   | 1   | 0     | 0    |
| Green | 0   | 1     | 0    |
| Blue  | 0   | 0     | 1    |

If you choose to use *n-1* dummy variables to avoid multicollinearity, you might drop one of the columns. If you drop the "Blue" column, the absence of "Red" and "Green" (i.e., both being 0) would imply "Blue".

### Key Points:

- **Advantages**:
    - Makes categorical data usable in models that require numerical input variables.
    - Helps in revealing more granular patterns in the relationship between features and response variables.
  
- **Disadvantages**:
    - Can lead to a substantial increase in dataset dimensions, especially with high-cardinality categorical variables.
    - Can introduce multicollinearity if not handled correctly.

- **Alternatives**:
    - While dummy variables are a popular way to handle categorical data, there are other methods like ordinal encoding, binary encoding, and embedding layers (for deep learning models).

In `pandas`, the `pd.get_dummies()` function can be used to easily convert categorical variables into dummy/indicator variables.

In [ ]:
df2 = pd.DataFrame([['apple', 6.5, 3.0, 0.5], ['apple', np.nan, np.nan, 1.5],
                     ['banana', np.nan, np.nan, np.nan], ['orange', 6.5, 3., 3.5],
                     ['orange', 6.5, 3.0, 3.5]],
                 columns=['A','B','C','D'])

In [ ]:
df2

In [ ]:
pd.get_dummies(df2, drop_first=True)

---
## Combining categories or other encodings

Instead of just using dummy variables, sometimes we might want to use an ordinal or other type of encoding.  One way to do this would be to create a mapping dictionary.  

```
mapping_dict = {old_value:new_value, 
another_old:another_new}

## apply
df['category'].map(mapping_dict)
```

In [ ]:
mapping = {'apple': 'Group1', 'banana': 'Group2', 'orange': 'Group1'}
df2['New'] = df2['A'].map(mapping)
df2

---

## Changing data types (casting)

Casting is the process of converting one data type into another.  Changing strings to numbers and numbers to strings is a common occurrence when cleaning data. Some pandas functions and methods used for casting include:


1. **astype()**:
    - The primary method to cast a pandas object to a specified data type.
    ```python
    df['column_name'] = df['column_name'].astype('new_data_type')
    ```

2. **to_numeric()**:
    - Convert a Series or a single column of a DataFrame to a numeric data type (either integer or float).
    - Useful for converting columns that might have been read as strings or objects due to the presence of non-numeric values.
    ```python
    df['column_name'] = pd.to_numeric(df['column_name'], errors='coerce')
    ```

3. **to_datetime()**:
    - Convert a Series or column to a datetime data type.
    ```python
    df['date_column'] = pd.to_datetime(df['date_column'])
    ```

4. **to_timedelta()**:
    - Convert a Series or column to a timedelta data type.
    ```python
    df['duration_column'] = pd.to_timedelta(df['duration_column'])
    ```

5. **pd.Categorical()**:
    - Convert a Series or column to a categorical data type.
    ```python
    df['category_column'] = pd.Categorical(df['category_column'])
    ```

When casting variables, it's essential to be aware of potential data loss or errors. For instance, casting a floating-point column to an integer will truncate the decimal parts, and trying to convert non-numeric strings to integers or floats can raise errors. Always check the data after casting to ensure that the conversion has been done correctly and as expected.

In [ ]:
df3 = pd.DataFrame([['apple', '6.5', 3.0, 0.5], ['apple', np.nan, np.nan, 1.5],
                     ['banana', np.nan, np.nan, np.nan], ['orange', 6.5, 3., 3.5],
                     ['orange', 6.5, 3.0, 3.5]],
                 columns=['A','B','C','D'])

In [ ]:
df3

In [ ]:
df3.info()

In [ ]:
df3['B'] = df3['B'].astype(float)

In [ ]:
df3.info()

---

## Unpacking

Many times, a single column will contain multiple pieces of information.  Learning how to extract or unpack this information is a common data cleaning task. 

Python's built in string methods and regular expressions are very helpful in these instances.  (See Python for Data Analysis Section 7.4)

### Helpful built in string methods

- `join`: Use string a delimiter for concatenating a sequence of other strings
- `strip`: Trim whitespace - Can also remove characters in the `chars` argument
- `split`: Split into a list of substrings based white space or `sep` argument
- `lower`: Converts to lowercase
- `replace`: Replace substring with another substring

### Other functions

Other functions, attributes, and methods that are commonly used in unpacking tasks include

- `.apply`: Use to pally a function along the axis (either rows or columns) of a DataFrame or on the elements of a Series.
- `lambda` functions
- Accessor objects allow you to perform operations on a Series without the need of loops    
    - `.str`: An accessor object used for vectorized string operations on a pandas Series
    - `.dt`: An accessor object used for vectorized date operations on a pandas datetime Series
    - *Further explanation:*   String and regular expression methods/functions  only work on `string` objects.  Date function/methods only work on `datetime` objects.  When dealing with pandas, the *value* inside a Series might be a `string` or `datetime`, but the Series itself is **not** a `string`.  Calling `.str` or .`dt` off of a Series allows `string` and `datetime` functions to be used on the Series.
    ```python
    s = pd.Series(['apple', 'banana', 'cherry'])    
    s.str[:3]
    s.str.startswith('a')

    dates = pd.Series(pd.date_range("2022-01-01", periods=3, freq="Y"))
    dates.dt.year
    ```



---
# Tidy Data
---

## Melting: wide to long

In [ ]:
import pandas as pd

In [ ]:
pew = pd.read_csv('Data/pew-raw.csv')
pew.head(2)

In [ ]:
pew.columns

In [ ]:
# function
pd.melt(pew,id_vars='religion')

In [ ]:
# method
pew = pew.melt(id_vars='religion')

In [ ]:
#pew

In [ ]:
pew.head()

In [ ]:
varnames = [' <$10k', ' $10-20k', '$20-30k', '$30-40k', ' $40-50k',
       '$50-75k']

In [ ]:
#pd.melt(pew, id_vars='religion',value_vars=varnames)

In [ ]:
bb.columns

In [ ]:
bb = pd.read_csv('Data/billboard.csv', encoding='latin1')

In [ ]:
bb.head()

In [ ]:
pd.melt(bb, id_vars=['artist','track'],value_vars=bb.columns[7:],
        var_name='week', value_name='rank')

In [ ]:
pd.melt(bb, id_vars=bb.columns[0:7], var_name='week', value_name='rank')

## Pivoting:  long to wide

In [ ]:
st = pd.read_csv('Data/stocks.csv')

In [ ]:
st

In [ ]:
temp_df = st.pivot(index='date', columns='symbol', values=['open','close'])

In [ ]:
temp_df

In [ ]:
temp_df.index

In [ ]:
temp_df.columns

In [ ]:
temp_df.reset_index()

In [ ]:
st.pivot(index='date',columns='symbol',values=['open','close']).reset_index()

In [ ]:
st = st.iloc[:,0:3]

In [ ]:
st

In [ ]:
wide_table = pd.pivot(st, index='date', columns='symbol')

In [ ]:
wide_table.columns

In [ ]:
wide_table.index

In [ ]:
wide_table.reset_index()

In [ ]:
st_new = pd.pivot(st, index='date', columns='symbol', values=['open'])

In [ ]:
st_new.reset_index()

In [ ]:
st_new.info()

In [ ]:
st_new.reset_index()

In [ ]:
country = pd.read_csv('Data/country_data2.csv')

In [ ]:
country.head(10)

In [ ]:
#pd.pivot(country, index=['Country','Region'], columns='Variable')

In [ ]:
pd.pivot_table(country, index=['Country','Region'], columns='Variable', values='Value',aggfunc='mean').reset_index()

In [ ]:
a = pd.pivot_table(country,index=['Country','Region'], 
         columns='Variable', values='Value').reset_index()

In [ ]:
a.index

In [ ]:
pd.pivot_table(country,index=['Country'], 
         columns=['Region','Variable'], values='Value').reset_index()